In [1]:
import pandas as pd

with open("./avg-test-result-in-paper", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, engine='python')
df.head()

,file_system,dup_rate,num_job,bandwidth(MiB/s)
0,Light-Dedup,0,1,808
1,Light-Dedup,0,2,1263
2,Light-Dedup,0,4,1666
3,Light-Dedup,0,8,1787
4,Light-Dedup,0,16,1573


- How much Light-Dedup outperform other deduplication file systems under 0, 25, 50, 75 duplication ratio?

In [2]:
fs_names = df['file_system'].drop_duplicates().reset_index().drop(columns=['index'])['file_system']

light_dedup =  df[df["file_system"] == "Light-Dedup"].copy().reset_index()

for fs_name in fs_names:
    if fs_name == "Light-Dedup":
        continue
    fs = df[df["file_system"] == fs_name]
    # print(pd.DataFrame([1, 2, 3]) / pd.DataFrame([1, 2, 3]))
    light_dedup[fs_name] = light_dedup["bandwidth(MiB/s)"] / fs["bandwidth(MiB/s)"].reset_index()["bandwidth(MiB/s)"]

light_dedup
    

,index,file_system,dup_rate,num_job,bandwidth(MiB/s),Light-Dedup(SHA256),NV-Dedup,NOVA
0,0,Light-Dedup,0,1,808,2.916968,1.034571,0.960761
1,1,Light-Dedup,0,2,1263,2.396584,0.991366,0.916546
2,2,Light-Dedup,0,4,1666,1.824754,1.007864,0.951456
3,3,Light-Dedup,0,8,1787,1.580018,1.172572,0.953575
4,4,Light-Dedup,0,16,1573,1.170387,1.189864,1.021429
5,20,Light-Dedup,25,1,740,2.681159,1.459566,0.857474
6,21,Light-Dedup,25,2,1306,2.516378,1.439912,0.971726
7,22,Light-Dedup,25,4,1990,2.068607,1.373361,1.139096
8,23,Light-Dedup,25,8,2127,1.443992,1.293009,1.136218
9,24,Light-Dedup,25,16,1600,0.956938,1.109570,1.039636


- How much throughput Light-Dedup achieves over NOVA and NV-Dedup when dedup ratio >= 25%?

In [3]:
print("Max higher than NV-Dedup:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Min higher than NV-Dedup:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Max higher than NOVA:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))
print("Min higher than NOVA:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))

Max higher than NV-Dedup:2.223082881487219
Min higher than NV-Dedup:1.1095700416088765
Max higher than NOVA:2.2324503311258277
Min higher than NOVA:0.8574739281575898


- How much slower Light-Dedup than NOVA when dedup ratio == 0%?

In [4]:
light_dedup[light_dedup["dup_rate"] == 0]["NOVA"]

0    0.960761
1    0.916546
2    0.951456
3    0.953575
4    1.021429
Name: NOVA, dtype: float64

- How much faster Light-Dedup than NV-Dedup when dedup ratio == 75%?

In [5]:
light_dedup[light_dedup["dup_rate"] == 75]["NV-Dedup"]

15    2.056931
16    2.207042
17    2.223083
18    1.941040
19    1.319374
Name: NV-Dedup, dtype: float64

- How much faster Light-Dedup than SHA256 when dedup ratio == 75%?

In [6]:
light_dedup[(light_dedup["dup_rate"] == 75) & (light_dedup["num_job"] == 1)]["Light-Dedup(SHA256)"]

15    2.233871
Name: Light-Dedup(SHA256), dtype: float64

- Performance Improvement of Light-Dedup from 0% to 75% 

In [7]:
light_dedup[(light_dedup["dup_rate"] == 75)].reset_index()["bandwidth(MiB/s)"]/light_dedup[(light_dedup["dup_rate"] == 0)].reset_index()["bandwidth(MiB/s)"]

0    1.028465
1    1.240697
2    1.722689
3    2.339675
4    2.143039
Name: bandwidth(MiB/s), dtype: float64